In [23]:
import pandas as pd

In [24]:
df = pd.read_csv("Taxonomy Freq.csv")

In [25]:
df[['kingdom', 'phylum', 'class',
    'order', 'family', 'genus', 'species']] = df[['kingdom', 
                                                         'phylum', 'class', 'order', 'family', 'genus', 'species']].fillna(value="Unknown")



In [27]:
allLevels = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'end']
taxLevel = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'end']
df = df.drop((set(allLevels) - set(taxLevel)), 1)

def getDict(taxLevelIndex, prevIndex):
    list_dicts = []
    if taxLevel[taxLevelIndex] == 'end':
        return ([], 0)
    if taxLevel[taxLevelIndex] == 'kingdom':
        df_size = df.shape[0]
        gp = df.groupby(taxLevel[taxLevelIndex])
    else:
        df_size = df[df[taxLevel[taxLevelIndex-1]] == prevIndex].shape[0]
        gp = df[df[taxLevel[taxLevelIndex-1]] == prevIndex].groupby(taxLevel[taxLevelIndex])
        
    indexes = gp.size().index.to_list()
    values = list(gp.size().values)
    num_types = len(gp)
    ind_val = zip(indexes, values)
    ind_val.sort(key=lambda tup: tup[1], reverse = True)
    for i in ind_val:
        index = i[0]
        value = i[1]
        next_data, num_types_next = getDict(taxLevelIndex + 1, index)
        list_dicts.append({"name": index, "taxLevel": taxLevel[taxLevelIndex], "types": num_types_next, "size": value, "ratio": (1.0 * value)/df_size, "children": next_data})
    
    
    return (list_dicts, num_types)
    
    #return a dictionary of name and children
    

In [29]:
json_dict, num_types = getDict(0, "blah")
json_dict = {"name": "Organisms", 'taxLevel': "organisms", "types": num_types, "children": json_dict, "ratio": 1.0, "size": df.shape[0]}

In [30]:
import json

with open('map_django/biodivmap/static/taxontree/treedata.json', 'w') as fp:
    json.dump(json_dict, fp)